<a href="https://colab.research.google.com/github/epadam/Machine-Learning-Tutorial-Demo-Resources/blob/master/notebooks/nlp/question%20anwsering/Haystack_QA_IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install farm-haystack

     |████████████████████████████████| 200 kB 30.5 MB/s 
     |████████████████████████████████| 321 kB 54.8 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 402 kB 51.9 MB/s 
     |████████████████████████████████| 8.4 MB 36.9 MB/s 
     |████████████████████████████████| 100 kB 10.4 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 3.8 MB 59.2 MB/s 
     |████████████████████████████████| 85 kB 972 kB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 332 kB 62.6 MB/s 
     |████████████████████████████████| 981 kB 67.4 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 204 kB 60.8 MB/s 
     |████████████████████████████████| 3.0 MB 49.7 MB/s 
     |██████████████████████

In [2]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [3]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

In [4]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [5]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

09/18/2021 23:27:58 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.108s]
09/18/2021 23:27:59 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.414s]
09/18/2021 23:27:59 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.226s]


In [6]:
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

09/18/2021 23:33:21 - INFO - haystack.preprocessor.utils -   Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
100%|██████████| 1095120/1095120 [00:00<00:00, 2065897.17B/s]


True

In [7]:
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

09/18/2021 23:38:06 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/331_Bran_Stark.txt
09/18/2021 23:38:06 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/515_The_Door__Game_of_Thrones_.txt
09/18/2021 23:38:06 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/368_Jaime_Lannister.txt
09/18/2021 23:38:06 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/76_Valar_Morghulis.txt
09/18/2021 23:38:06 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/195_World_of_A_Song_of_Ice_and_Fire.txt
09/18/2021 23:38:06 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/229_Game_of_Thrones.txt
09/18/2021 23:38:06 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/514_Book_of_the_Stranger.txt
09/18/2021 23:38:06 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/26_Game_of_Thrones__Season_3__soundtrack_.txt
09/18/2021 23:38

In [8]:
print(dicts[:3])

[{'text': '\'\'\'Brandon Stark\'\'\', typically called \'\'\'Bran\'\'\', is a fictional character in the \'\'A Song of Ice and Fire\'\' series of epic fantasy novels by American author George R. R. Martin, and its television adaptation \'\'Game of Thrones\'\'.\nIntroduced in 1996\'s \'\'A Game of Thrones\'\', Bran is the second son and fourth child of Lord Eddard and Lady Catelyn Stark of Winterfell, the ancient capital of the North of the fictional kingdom of Westeros.  He subsequently appeared in \'\'A Clash of Kings\'\' (1998) and \'\'A Storm of Swords\'\' (2000), was one of a few prominent characters that were not included in the fourth novel \'\'A Feast for Crows\'\' (2005), but returned in the fifth novel \'\'A Dance with Dragons\'\' (2011).  Bran dreams of becoming a knight since childhood, but is rendered paraplegic by Jaime Lannister early into the first novel after stumbling upon the latter\'s incestuous affair with twin sister Cersei Lannister.  Awaking from a months-long co

In [9]:
document_store.write_documents(dicts)

09/18/2021 23:38:53 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.846s]
09/18/2021 23:38:54 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.137s]
09/18/2021 23:38:55 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.867s]
09/18/2021 23:38:57 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.095s]
09/18/2021 23:38:58 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.090s]


In [10]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [11]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

09/19/2021 00:02:40 - INFO - farm.utils -   Using device: CPU 
09/19/2021 00:02:40 - INFO - farm.utils -   Number of GPUs: 0
09/19/2021 00:02:40 - INFO - farm.utils -   Distributed Training: False
09/19/2021 00:02:40 - INFO - farm.utils -   Automatic Mixed Precision: None
09/19/2021 00:02:40 - INFO - filelock -   Lock 140208729993552 acquired on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

09/19/2021 00:02:41 - INFO - filelock -   Lock 140208729993552 released on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock
09/19/2021 00:02:41 - INFO - filelock -   Lock 140208743840848 acquired on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

09/19/2021 00:02:55 - INFO - filelock -   Lock 140208743840848 released on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock
Some weights of the model checkpoint at deepset/roberta-base-squad2 were not used when initializing RobertaModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['ro

...

09/19/2021 00:03:01 - INFO - filelock -   Lock 140208730041232 released on /root/.cache/huggingface/transformers/b87d46371731376b11768b7839b1a5938a4f77d6bd2d9b683f167df0026af432.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
09/19/2021 00:03:01 - INFO - filelock -   Lock 140208729163728 acquired on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

09/19/2021 00:03:01 - INFO - filelock -   Lock 140208729163728 released on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock
09/19/2021 00:03:02 - INFO - filelock -   Lock 140208728437648 acquired on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

09/19/2021 00:03:02 - INFO - filelock -   Lock 140208728437648 released on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock
09/19/2021 00:03:02 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
09/19/2021 00:03:02 - INFO - farm.utils -   Using device: CPU 
09/19/2021 00:03:02 - INFO - farm.utils -   Number of GPUs: 0
09/19/2021 00:03:02 - INFO - farm.utils -   Distributed Training: False
09/19/2021 00:03:02 - INFO - farm.utils -   Automatic Mixed Precision: None
09/19/2021 00:03:02 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
09/19/2021 00:03:02 - INFO - farm.infer -    0    0 
09/19/2021 00:03:02 - INFO - farm.infer -   /w\  /w\
09/19/2021 00:03:02 - INFO - farm.infer -   /'\  / \
09/19/2021 00:03:02 - INFO - farm.infer -     
09/19/2021 00:03:02 - INFO - farm.utils -   Usin

In [12]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [13]:
prediction = pipe.run(
    query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

09/19/2021 00:03:51 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.188s]
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.73s/ Batches]


In [14]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's "
                   'half-brother Jon Snow gifts A'},
    {   'answer': 'Lord Eddard Stark',
        'context': 'ark daughters.\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.'},
    {   'answer': 'Eddard "Ned" Stark',
        'context': '=\n'
                   "After Varys tells him that Sansa Stark's life is also at "
                   'stake, Eddard "Ned" S